In [20]:
#open drone_occupancy.json
import json
drone_paths = {}
with open('drone_occupancy.json') as f:
    data = json.load(f)
    print(data)
    number_of_drones = 3
    for i in range(1,number_of_drones+1):
        drone_paths[i] = []
        for j in range(len(data)):
            for k in range(len(data[j])):
                for l in range(len(data[j][k])):
                    drones_in_cell = data[j][k][l]
                    for drone_tag,time_stamp in drones_in_cell:
                        if drone_tag == i:
                            drone_paths[i].append((j,k,l,time_stamp))
        #sort drone_paths[i] by time_stamp
        drone_paths[i].sort(key=lambda tup: tup[3])
    print(drone_paths)
            
def check_overlap(obj):
    """
    Check if there are any overlapping timestamps for drones in the same coordinates.

    Args:
        obj (dict): Dictionary where keys are drone tags and values are lists of tuples (x, y, z, timestamp).

    Returns:
        bool: True if there are no overlaps, False otherwise.
    """
    coordinates_timestamps = {}  # Dictionary to store coordinates and their timestamps

    for drone_tag, positions in obj.items():
        for x, y, z, timestamp in positions:
            # Check if the coordinates are already in the dictionary
            if (x, y, z) in coordinates_timestamps:
                # Check if the timestamp overlaps with existing timestamps for the same coordinates
                if timestamp in coordinates_timestamps[(x, y, z)]:
                    return True  # Overlapping timestamps found
                else:
                    coordinates_timestamps[(x, y, z)].append(timestamp)
            else:
                # If coordinates are not in the dictionary, add them with the current timestamp
                coordinates_timestamps[(x, y, z)] = [timestamp]

    return False  # No overlapping timestamps found

check_overlap(drone_paths)

[[[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [

False

In [4]:
def get_points_in_between(point1, point2):
    points = []
    num_points = max(abs(point1[0] - point2[0]), abs(point1[1] - point2[1]), abs(point1[2] - point2[2]))
    for t in range(1,num_points):
        t_normalized = t / num_points
        x = int(point1[0] + t_normalized * (point2[0] - point1[0]))
        y = int(point1[1] + t_normalized * (point2[1] - point1[1]))
        z = int(point1[2] + t_normalized * (point2[2] - point1[2]))
        points.append((x, y, z))
    return points

get_points_in_between((1,1,1),(3,3,3))

[(2, 2, 2)]

In [16]:
tolerance = 1.0

def euclidean_distance(start_point, end_point):
    return ((start_point[0] - end_point[0]) ** 2 + (start_point[1] - end_point[1]) ** 2 + (start_point[2] - end_point[2]) ** 2) ** 0.5

def douglas_peucker(points):
    """
    Applys douglas_peucker algorithm on generated points from bfs to lower the number of control points

    Args:
        points (list): A list of all paths generated by BFS.

    Returns:
        list: similar path with fewer number of control points.
    """
    if len(points) < 2:
        return [points[0], points[1]]
    
    if len(points) <= 2:
        return [points[0], points[-1]]

    # Find the point farthest from the line connecting the start and end points
    max_distance = 0
    max_index = 0
    start, end = points[0], points[-1]

    for i in range(1, len(points) - 1):
        distance = euclidean_distance(points[i], start) + euclidean_distance(points[i], end)
        line_distance = euclidean_distance(start, end)
        d = distance - line_distance

        if d > max_distance:
            max_distance = d
            max_index = i

    if max_distance <= tolerance:
        return [start, end]
    # Recursively simplify the path
    first_segment = douglas_peucker(points[:max_index + 1])
    second_segment = douglas_peucker(points[max_index:])

    return first_segment[:-1] + second_segment  # Exclude the duplicated point

douglas_peucker([(20, 20, 20), (20, 20, 17), (18, 13, 13), (18, 18, 14), (18, 12, 12)])


[(20, 20, 20), (20, 20, 17), (18, 13, 13), (18, 18, 14), (18, 12, 12)]